In [2]:
!pip install -r requirements.txt

In [11]:
import time
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials (1).env")
deployment_name = "gpt-35-turbo-16k"

# Create an OpenAI Azure client
client = AzureOpenAI(
    api_key=os.getenv("5730f831d7834af381c41ff62a3c3f30"),#"8b96d7ba6a31403089100421919c7962",  #change to your API KEY Value
    api_version="2024-02-15-preview",
    azure_endpoint =os.getenv("https://openai-us-sa.openai.azure.com/") #change to your API ENDPOINT
    )

# Create an Asssitant with the code_interpreter tool
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=deployment_name,
)

# Create a Thread
thread = client.beta.threads.create()

# Create a Message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

# Create a Run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

# Check the status of a Run
print("checking assistant status. ")
while True:
    # Retrieve a Run by Thread and Run id
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    
    if run.status == "completed":
        # Get the messages for a thread
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        # Print the messages
        print("messages: ")
        for message in messages:
            assert message.content[0].type == "text"
            print({"role": message.role, "message": message.content[0].text.value})
        # Dispose of the assistant
        client.beta.assistants.delete(assistant.id)
        # Dispose of the thread
        client.beta.threads.delete(thread.id)
        
        break
    elif run.status == "requires_action":
        # handle function calling and continue with the execution
        pass
    elif run.status == "expired" or run.status == "failed" or run.status == "cancelled":
        # run failed, expired, or was cancelled
        break    
    else:
        print("in progress...")
        time.sleep(5)

OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.